In [6]:
def min_max_scaling(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# Sample line of data
line = "1 0.49297499999999994 0.4549378453038674 0.47575 0.29722375690607733 300.49 298.29 2.0 153.06 424.53 2.0 382.51 221.78 2.0 157.46 436.97 2.0 438.51 421.4 2.0"

# Extract all px, py coordinates from the provided data
keypoint_coordinates = [float(coord) for coord in line.split()[5:]]
min_val, max_val = min(keypoint_coordinates), max(keypoint_coordinates)

# Normalize px, py coordinates using Min-Max Scaling
normalized_keypoints = []
for i in range(5, len(line.split()), 3):
    px = float(line.split()[i])
    py = float(line.split()[i+1])
    visibility = float(line.split()[i+2])
    
    normalized_px = min_max_scaling(px, min_val, max_val)
    normalized_py = min_max_scaling(py, min_val, max_val)
    
    normalized_keypoints.append((normalized_px, normalized_py, visibility))

    # Print statements for debugging
    print(f"Original px: {px}, Original py: {py}")
    print(f"Normalized px: {normalized_px}, Normalized py: {normalized_py}")

print(normalized_keypoints)


Original px: 300.49, Original py: 298.29
Normalized px: 0.6838102219880415, Normalized py: 0.6787702458133835
Original px: 153.06, Original py: 424.53
Normalized px: 0.3460630913381137, Normalized py: 0.9679732423083091
Original px: 382.51, Original py: 221.78
Normalized px: 0.8717096973723397, Normalized py: 0.5034936198483425
Original px: 157.46, Original py: 436.97
Normalized px: 0.3561430436874299, Normalized py: 0.9964720166777394
Original px: 438.51, Original py: 421.4
Normalized px: 1.0, Normalized py: 0.9608027307507273
[(0.6838102219880415, 0.6787702458133835, 2.0), (0.3460630913381137, 0.9679732423083091, 2.0), (0.8717096973723397, 0.5034936198483425, 2.0), (0.3561430436874299, 0.9964720166777394, 2.0), (1.0, 0.9608027307507273, 2.0)]


In [7]:
def min_max_scaling(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# Sample line of data
line = "1 0.49297499999999994 0.4549378453038674 0.47575 0.29722375690607733 300.49 298.29 2.0 153.06 424.53 2.0 382.51 221.78 2.0 157.46 436.97 2.0 438.51 421.4 2.0"

# Extract all coordinates and visibility from the provided data
coordinates_and_visibility = [float(coord) for coord in line.split()[1:]]

# Separate the values into different categories
class_index, x, y, width, height = coordinates_and_visibility[:5]
keypoint_data = coordinates_and_visibility[5:]

# Calculate min and max values for normalization
min_val, max_val = min(coordinates_and_visibility), max(coordinates_and_visibility)

# Normalize bounding box coordinates using Min-Max Scaling
normalized_x = min_max_scaling(x, min_val, max_val)
normalized_y = min_max_scaling(y, min_val, max_val)
normalized_width = min_max_scaling(width, min_val, max_val)
normalized_height = min_max_scaling(height, min_val, max_val)

normalized_bounding_box = (normalized_x, normalized_y, normalized_width, normalized_height)

# Normalize keypoint coordinates using Min-Max Scaling
normalized_keypoints = []
for i in range(0, len(keypoint_data), 3):
    px = keypoint_data[i]
    py = keypoint_data[i+1]
    visibility = keypoint_data[i+2]

    normalized_px = min_max_scaling(px, min_val, max_val)
    normalized_py = min_max_scaling(py, min_val, max_val)

    normalized_keypoints.append((normalized_px, normalized_py, visibility))

# Assemble the normalized YOLO data
normalized_yolo_data = [class_index, normalized_x, normalized_y, normalized_width, normalized_height] + [coord for kp in normalized_keypoints for coord in kp]

print(normalized_yolo_data)


IndexError: list index out of range

In [8]:
import os
import json

def json_to_yolo(json_data, image_width, image_height):
    yolo_data = []

    for annotation in json_data['annotations']:
        class_index = annotation['category_id']
        x, y, width, height = annotation['bbox']

        # Normalize bounding box coordinates
        normalized_x = x / image_width
        normalized_y = y / image_height
        normalized_width = width / image_width
        normalized_height = height / image_height

        yolo_line = f"{class_index} {normalized_x} {normalized_y} {normalized_width} {normalized_height}"

        # Normalize keypoint coordinates
        keypoints = annotation['keypoints']
        for i in range(0, len(keypoints), 3):
            px, py, visibility = keypoints[i], keypoints[i+1], keypoints[i+2]

            normalized_px = px / image_width
            normalized_py = py / image_height

            yolo_line += f" {normalized_px} {normalized_py} {visibility}"

        yolo_data.append(yolo_line)

    return yolo_data

def process_json_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            json_file_path = os.path.join(folder_path, filename)

            with open(json_file_path, 'r') as json_file:
                json_data = json.load(json_file)

                # Modify image_width and image_height based on your dataset
                image_width = json_data['images'][0]['width']
                image_height = json_data['images'][0]['height']

                yolo_data = json_to_yolo(json_data, image_width, image_height)

                # Save YOLO-formatted data to a file
                yolo_file_path = os.path.join(folder_path, f"{os.path.splitext(filename)[0]}.txt")
                with open(yolo_file_path, 'w') as yolo_file:
                    for line in yolo_data:
                        yolo_file.write(line + '\n')

if __name__ == "__main__":
    folder_path = input("Enter the folder path containing JSON files:  ")
    process_json_files(folder_path)
